In [3]:
import pandas as pd
import os
import psutil

# Step 1: Define Data Path
data_path = "D:/mimic-iv-3.1"

# Step 2: Check memory usage before loading data
def check_memory():
    print(f"Memory Usage: {psutil.virtual_memory().percent}%")
check_memory()


Memory Usage: 42.1%


In [289]:
# Create an empty DataFrame with the key columns
admissions = pd.DataFrame(columns=["subject_id", "hadm_id"])
diagnosis_count = pd.DataFrame(columns=["subject_id", "hadm_id"])
diagnoses = pd.DataFrame(columns=["subject_id", "hadm_id"])

In [291]:
# Step 3: Load MIMIC-IV Tables with optimized memory handling
# Load small tables directly
admissions = pd.read_csv(os.path.join(data_path, "hosp/admissions.csv.gz"), 
                          usecols=["subject_id", "hadm_id", "admission_type", "admission_location", "discharge_location", "admittime", "dischtime","insurance", "race", "marital_status", "edregtime", "edouttime"], 
                          low_memory=False)

# Calculate time spent in ED before admission
admissions["ed_time_spent"] = (pd.to_datetime(admissions["edouttime"]) - pd.to_datetime(admissions["edregtime"])).dt.total_seconds() / 60  # Convert to minutes

# Drop original ED timestamps
admissions = admissions.drop(columns=["edregtime", "edouttime"], errors="ignore")

# Compute hospital length of stay (LOS) in days
admissions["los_hosp"] = (pd.to_datetime(admissions["dischtime"]) - pd.to_datetime(admissions["admittime"])).dt.total_seconds() / (60 * 60 * 24)

# Drop original timestamps
admissions = admissions.drop(columns=["admittime", "dischtime"], errors="ignore")


In [7]:
print(admissions.head())  # Show first 5 rows

   subject_id   hadm_id  admission_type      admission_location  \
0    10000032  22595853          URGENT  TRANSFER FROM HOSPITAL   
1    10000032  22841357        EW EMER.          EMERGENCY ROOM   
2    10000032  25742920        EW EMER.          EMERGENCY ROOM   
3    10000032  29079034        EW EMER.          EMERGENCY ROOM   
4    10000068  25022803  EU OBSERVATION          EMERGENCY ROOM   

  discharge_location insurance marital_status   race  ed_time_spent  los_hosp  
0               HOME  Medicaid        WIDOWED  WHITE          253.0  0.786111  
1               HOME  Medicaid        WIDOWED  WHITE          337.0  1.015278  
2            HOSPICE  Medicaid        WIDOWED  WHITE          286.0  1.754167  
3               HOME  Medicaid        WIDOWED  WHITE          486.0  2.222222  
4                NaN       NaN         SINGLE  WHITE          511.0  0.298611  


In [9]:
 # Load patient demographics
patients = pd.read_csv(os.path.join(data_path, "hosp/patients.csv.gz"),
                        usecols=["subject_id", "gender", "anchor_age", "anchor_year"],
                        low_memory=False)


In [11]:
print(patients.head())

   subject_id gender  anchor_age  anchor_year
0    10000032      F          52         2180
1    10000048      F          23         2126
2    10000058      F          33         2168
3    10000068      F          19         2160
4    10000084      M          72         2160


In [293]:
# Load diagnoses and count unique conditions per admission
diagnoses = pd.read_csv(os.path.join(data_path, "hosp/diagnoses_icd.csv.gz"),
                         usecols=["subject_id", "hadm_id", "icd_code"],
                         low_memory=False)

diagnosis_counts = diagnoses.groupby(["subject_id", "hadm_id"]).agg(
    num_comorbidities=('icd_code', 'nunique'),  # Count unique diagnoses
    diagnosis_list=('icd_code', lambda x: list(x))  # Retain list of all diagnoses for admission
).reset_index()

In [295]:
print(diagnoses.head())

   subject_id   hadm_id icd_code
0    10000032  22595853     5723
1    10000032  22595853    78959
2    10000032  22595853     5715
3    10000032  22595853    07070
4    10000032  22595853      496


In [297]:
print(diagnosis_counts.head())

   subject_id   hadm_id  num_comorbidities  \
0    10000032  22595853                  8   
1    10000032  22841357                  8   
2    10000032  25742920                 10   
3    10000032  29079034                 13   
4    10000068  25022803                  1   

                                      diagnosis_list  
0  [5723, 78959, 5715, 07070, 496, 29680, 30981, ...  
1   [07071, 78959, 2875, 2761, 496, 5715, V08, 3051]  
2  [07054, 78959, V462, 5715, 2767, 2761, 496, V0...  
3  [45829, 07044, 7994, 2761, 78959, 2767, 3051, ...  
4                                            [30500]  


In [25]:
# Load ICU stays with length of stay
icustays = pd.read_csv(os.path.join(data_path, "icu/icustays.csv.gz"),
                        usecols=["subject_id", "hadm_id", "stay_id", "last_careunit", "los"],
                        low_memory=False)

# Count previous ICU admissions per patient
icu_history = icustays.groupby("subject_id")["stay_id"].count().reset_index()
icu_history.rename(columns={"stay_id": "prior_icu_admissions"}, inplace=True)


In [27]:
print(icustays.head())

   subject_id   hadm_id   stay_id                        last_careunit  \
0    10000032  29079034  39553978   Medical Intensive Care Unit (MICU)   
1    10000690  25860671  37081114   Medical Intensive Care Unit (MICU)   
2    10000980  26913865  39765666   Medical Intensive Care Unit (MICU)   
3    10001217  24597018  37067082  Surgical Intensive Care Unit (SICU)   
4    10001217  27703517  34592300  Surgical Intensive Care Unit (SICU)   

        los  
0  0.410266  
1  3.893252  
2  0.497535  
3  1.118032  
4  0.948113  


In [37]:
print(icu_history.head())

   subject_id  prior_icu_admissions
0    10000032                     1
1    10000690                     1
2    10000980                     1
3    10001217                     2
4    10001725                     1


In [85]:
chunksize = 200000
# List of lab item IDs for the desired tests
lab_ids = [
    "51300",  # WBC Count
    "51111",  # Hemoglobin
    "51265",  # Platelet Count
    "52623",  # Sodium
    "52610",  # Potassium
    "52535",  # Chloride
    "51739",  # Total CO2 (surrogate for bicarbonate)
    "52569",  # Glucose
    "52647",  # Urea Nitrogen (BUN)
    "52546",  # Creatinine
    "53084",  # ALT
    "53088",  # AST
    "53086",  # Alkaline Phosphatase
    "53089",  # Total Bilirubin
    "50889",  # C-Reactive Protein (CRP)
    "50862"   # Albumin
]   # these tests werent extracted as 

# ---   Process Lab Events from hosp/labevents.csv.gz ---
lab_chunks = []
lab_file = os.path.join(data_path, "hosp/labevents.csv.gz")
for chunk in pd.read_csv(
        lab_file,
        usecols=["subject_id", "hadm_id", "itemid", "charttime", "valuenum"],
        chunksize=chunksize,
        low_memory=False):
    # Filter for only the desired tests by itemid (cast to str for safe comparison)
    chunk = chunk[chunk["itemid"].astype(str).isin(lab_ids)]
    lab_chunks.append(chunk)

# Concatenate all filtered chunks
lab_events = pd.concat(lab_chunks, ignore_index=True)
lab_events.sort_values(["subject_id", "hadm_id", "charttime"], inplace=True)

# Group by (subject_id, hadm_id, itemid) and keep only the first recorded value per test
lab_first = lab_events.groupby(["subject_id", "hadm_id", "itemid"]).first().reset_index()

In [87]:
print(lab_events.head())

    subject_id     hadm_id  itemid            charttime  valuenum
4     10000032  22595853.0   51265  2180-05-07 05:05:00      71.0
9     10000032  22841357.0   51265  2180-06-27 05:10:00     137.0
10    10000032  22841357.0   50862  2180-06-27 05:10:00       2.7
18    10000032  25742920.0   51265  2180-08-06 06:36:00     133.0
19    10000032  25742920.0   50862  2180-08-06 06:36:00       3.5


In [105]:
# ---  Pivot the lab  data ---
# Pivot on itemid (which is unique) to avoid duplicate errors,

lab_pivot = lab_first.pivot(
    index=["subject_id", "hadm_id"],
    columns="itemid",
    values="valuenum"
).reset_index()
# rename columns using the abbreviation mapping
lab_pivot.rename(columns={
    50862: "Alb",    # Albumin: serum albumin concentration, a nutritional/inflammatory marker
    50889: "CRP",    # CRP: C-Reactive Protein, an acute-phase reactant indicating inflammation
    51111: "Hb",     # Hb: Hemoglobin, for oxygen-carrying capacity (assesses anemia)
    51265: "Plt",    # Plt: Platelet Count, for clotting function
    51300: "WBC"     # WBC: White Blood Cell Count, for immune/inflammatory status
}, inplace=True)
print(lab_pivot.shape)
print(lab_pivot.head())


(425393, 7)
itemid  subject_id     hadm_id  Alb  CRP  Hb    Plt  WBC
0         10000032  22595853.0  NaN  NaN NaN   71.0  NaN
1         10000032  22841357.0  2.7  NaN NaN  137.0  NaN
2         10000032  25742920.0  3.5  NaN NaN  133.0  NaN
3         10000032  29079034.0  3.8  NaN NaN   94.0  NaN
4         10000084  23052089.0  4.0  NaN NaN  263.0  NaN


In [111]:
chunksize = 200000

# 1. Read the d_items table to get item metadata, focusing on abbreviation, unit, etc.
d_items = pd.read_csv(
    os.path.join(data_path, "icu/d_items.csv.gz"),
    usecols=["itemid", "abbreviation", "category", "unitname", "param_type"]
)

# 2. Read chartevents in chunks, storing them in a list
chart_file = os.path.join(data_path, "icu/chartevents.csv.gz")
chart_chunks = []

for chunk in pd.read_csv(
    chart_file,
    usecols=["subject_id", "hadm_id", "stay_id", "itemid", "charttime", "valuenum"],
    chunksize=chunksize,
    low_memory=True
):
    chart_chunks.append(chunk)

# Concatenate all chunks
chartevents = pd.concat(chart_chunks, ignore_index=True)

# 3. Sort and group to keep only the first recorded value per (subject, hadm, itemid)
chartevents.sort_values(["subject_id", "hadm_id", "charttime"], inplace=True)
chartevents_first = chartevents.groupby(["subject_id", "hadm_id", "itemid"]).first().reset_index()

# 4. Merge with d_items at the end to attach abbreviations and other metadata
chartevents_first = chartevents_first.merge(d_items, on="itemid", how="left")

In [116]:
# 5. Pivot on the unique itemid to avoid duplicate abbreviation issues.
chartevents_pivot = chartevents_first.pivot(
    index=["subject_id", "hadm_id"],
    columns="itemid",
    values="valuenum"
).reset_index()

# 6. Create a mapping from itemid to abbreviation.
id_to_abbr = d_items.set_index("itemid")["abbreviation"].to_dict()

# 7. Rename the pivoted columns using the mapping.
chartevents_pivot.rename(columns=id_to_abbr, inplace=True)

print(chartevents_pivot.shape)
print(chartevents_pivot.head())

(85242, 2313)
itemid  subject_id   hadm_id  Problem List    HR  HR Alarm - High  \
0         10000032  29079034           NaN  91.0            120.0   
1         10000690  25860671           NaN  79.0            120.0   
2         10000980  26913865           NaN  77.0              NaN   
3         10001217  24597018           NaN  86.0            120.0   
4         10001217  27703517           NaN  96.0            120.0   

itemid  HR Alarm - Low  Heart Rhythm  ABPs  ABPd  ABPm  ...  \
0                 60.0           NaN   NaN   NaN   NaN  ...   
1                 50.0           NaN   NaN   NaN   NaN  ...   
2                  NaN           NaN   NaN   NaN   NaN  ...   
3                 60.0           NaN   NaN   NaN   NaN  ...   
4                 60.0           NaN   NaN   NaN   NaN  ...   

itemid  TUN SecurePort IV  Suction events  (L)  Internal FMS Care  \
0                     NaN                  NaN                NaN   
1                     NaN                  NaN        

In [240]:
chunksize = 200000

# List of item IDs to extract, based on your provided list
icu_item_ids = [
    # 1. Physiological Parameters
    "220045",  # Heart Rate
    "220052",  # Mean Arterial Blood Pressure
    "220210",  # Respiratory Rate
    "220277",  # Oxygen Saturation (pulseoxymetry)
    "223761",  # Temperature Fahrenheit
    "220739",  # GCS – Eye Opening
    "223900",  # GCS – Verbal Response
    "223901",  # GCS – Motor Response
    "228096",  # Richmond-RAS Scale

    # 2. Laboratory Results
    "220546",  # WBC
    "220228",  # Hemoglobin
    "227457",  # Platelet Count
    "220645",  # Sodium (serum)
    "227442",  # Potassium (serum)
    "220602",  # Chloride (serum)
    "227443",  # HCO3 (serum)
    "225624",  # BUN
    "220615",  # Creatinine (serum)
    "220621",  # Glucose (serum)
    "225668",  # Lactic Acid
    "223830",  # pH (Arterial)
    "220224",  # Arterial O2 pressure (PaO2)
    "220235",  # Arterial CO2 pressure (PaCO2)
    "224828",  # Arterial Base Excess

    # 3. Intervention-Based Variables
    "223849",  # Ventilator Mode
    "223836",  # Airway Type
    "223835",  # Inspired O2 Fraction (FiO2)
    "220339",  # PEEP set
    "227210",  # Propofol (Intubation)
    "227211",  # Ketamine (Intubation)
    "227213",  # Vecuronium (Intubation)
    "220120"   # Intra-Aortic Balloon Pump (IABP)
]

# Read the ICU chartevents in chunks and filter to only include our items of interest
chart_file = os.path.join(data_path, "icu/chartevents.csv.gz")
chart_chunks = []
for chunk in pd.read_csv(
    chart_file,
    usecols=["subject_id", "hadm_id", "stay_id", "itemid", "charttime", "valuenum"],
    chunksize=chunksize,
    low_memory=True
):
    # Convert itemid to string for safe matching, then filter
    chunk = chunk[chunk["itemid"].astype(str).isin(icu_item_ids)]
    chart_chunks.append(chunk)

# Concatenate all filtered chunks
icu_chartevents = pd.concat(chart_chunks, ignore_index=True)

#  sort by subject, admission, and charttime, then keep the first recorded value per (subject, hadm, itemid)
icu_chartevents.sort_values(["subject_id", "hadm_id", "charttime"], inplace=True)
icu_first = icu_chartevents.groupby(["subject_id", "hadm_id", "itemid"]).first().reset_index()

# Optional: Pivot so that each admission is one row with a column per itemid
icu_pivot = icu_first.pivot(
    index=["subject_id", "hadm_id"],
    columns="itemid",
    values="valuenum"
).reset_index()

print(icu_pivot.shape)
print(icu_pivot.head())

(85242, 31)
itemid  subject_id   hadm_id  220045  220052  220120  220210  220224  220228  \
0         10000032  29079034    91.0     NaN     NaN    24.0     NaN     NaN   
1         10000690  25860671    79.0     NaN     NaN    23.0   123.0     9.5   
2         10000980  26913865    77.0     NaN     NaN    23.0     NaN     NaN   
3         10001217  24597018    86.0     NaN     NaN    18.0     NaN    11.2   
4         10001217  27703517    96.0     NaN     NaN    11.0     NaN    12.3   

itemid  220235  220277  ...  223849  223900  223901  224828  225624  225668  \
0          NaN    98.0  ...     NaN     4.0     6.0     NaN    33.0     NaN   
1         54.0   100.0  ...     NaN     5.0     6.0     0.0    21.0     NaN   
2          NaN   100.0  ...    11.0     5.0     6.0     NaN     NaN     1.7   
3          NaN    99.0  ...     NaN     5.0     6.0     NaN     9.0     NaN   
4          NaN   100.0  ...     NaN     5.0     6.0     NaN    10.0     NaN   

itemid  227442  227443  227457  

In [248]:
id_to_label = {
    220045: "heart_rate",      # Heart Rate
    220052: "map",             # Mean Arterial Blood Pressure
    220210: "resp_rate",       # Respiratory Rate
    220277: "spo2",            # Oxygen Saturation
    223761: "temp",            # Temperature Fahrenheit
    220739: "gcs_eye",         # GCS – Eye Opening
    223900: "gcs_verbal",      # GCS – Verbal Response
    223901: "gcs_motor",       # GCS – Motor Response
    228096: "ras_scale",       # Richmond-RAS Scale
    220546: "wbc",             # White Blood Cell Count
    220228: "hb",              # Hemoglobin
    227457: "plt",             # Platelet Count
    220645: "sodium",          # Sodium (serum)
    227442: "potassium",       # Potassium (serum)
    220602: "chloride",        # Chloride (serum)
    227443: "hco3",            # HCO3 (serum)
    225624: "bun",             # BUN
    220615: "creatinine",      # Creatinine (serum)
    220621: "glucose",         # Glucose (serum)
    225668: "lactate",         # Lactic Acid
    223830: "ph",              # pH (Arterial)
    220224: "pao2",            # Arterial O2 pressure (PaO₂)
    220235: "paco2",           # Arterial CO₂ pressure (PaCO₂)
    224828: "base_excess",     # Arterial Base Excess
    223849: "vent_mode",       # Ventilator Mode
    223836: "airway",          # Airway Type
    223835: "fio2",            # Inspired O₂ Fraction (FiO₂)
    220339: "peep",            # PEEP set
    227210: "propofol",        # Propofol (Intubation)
    227211: "ketamine",        # Ketamine (Intubation)
    227213: "vecuronium",      # Vecuronium (Intubation)
    220120: "iabp"             # Intra-Aortic Balloon Pump (IABP)
}

icu_pivot.rename(columns=lambda col: "icu_" + id_to_label[col]
                                  if col not in ["subject_id", "hadm_id"] and col in id_to_label
                                  else col,
                 inplace=True)



In [250]:
print(icu_pivot.shape)
print(icu_pivot.head())

(85242, 31)
itemid  subject_id   hadm_id  icu_hr  icu_map  icu_iabp  icu_rr  icu_pao2  \
0         10000032  29079034    91.0      NaN       NaN    24.0       NaN   
1         10000690  25860671    79.0      NaN       NaN    23.0     123.0   
2         10000980  26913865    77.0      NaN       NaN    23.0       NaN   
3         10001217  24597018    86.0      NaN       NaN    18.0       NaN   
4         10001217  27703517    96.0      NaN       NaN    11.0       NaN   

itemid  icu_hb  icu_paco2  icu_spo2  ...  icu_vent_mode  icu_gcs_verbal  \
0          NaN        NaN      98.0  ...            NaN             4.0   
1          9.5       54.0     100.0  ...            NaN             5.0   
2          NaN        NaN     100.0  ...           11.0             5.0   
3         11.2        NaN      99.0  ...            NaN             5.0   
4         12.3        NaN     100.0  ...            NaN             5.0   

itemid  icu_gcs_motor  icu_base_excess  icu_bun  icu_lactate  icu_potassiu

In [120]:
# Load prescriptions and flag high-risk medications
prescriptions = pd.read_csv(os.path.join(data_path, "hosp/prescriptions.csv.gz"),
                            usecols=["subject_id", "hadm_id", "drug"],
                            low_memory=False)
# Standardize the drug names
prescriptions["drug"] = prescriptions["drug"].astype(str).str.lower().str.strip()

# Retain a list of all drugs per admission
all_drugs = prescriptions.groupby(["subject_id", "hadm_id"])["drug"].agg(list).reset_index()

# Define expanded high-risk medications list based on literature
high_risk_meds = [
    "lorazepam", "midazolam", "diazepam", "clonazepam", "alprazolam", "temazepam", "chlordiazepoxide",
    "morphine", "fentanyl", "hydromorphone", "oxycodone", "meperidine", "codeine", "tramadol", "buprenorphine",
    "propofol", "dexmedetomidine",
    "haloperidol", "olanzapine", "quetiapine", "risperidone", "ziprasidone", "chlorpromazine", "aripiprazole",
    "diphenhydramine", "hydroxyzine", "promethazine", "oxybutynin", "scopolamine", "benztropine", "atropine",
    "dexamethasone", "prednisone", "methylprednisolone", "hydrocortisone",
    "metoclopramide", "cyclobenzaprine", "baclofen", "amantadine"
]

# Flag high-risk medications
prescriptions["high_risk_med"] = prescriptions["drug"].apply(
    lambda x: 1 if any(med in x for med in high_risk_meds) else 0
)
 

In [150]:
# --- 4. Compute Metrics per Admission ---
# a) Unique count of high-risk medications per admission
unique_high_risk_med = prescriptions[prescriptions["high_risk_med"] == 1] \
    .groupby(["subject_id", "hadm_id"])["drug"] \
    .nunique().reset_index(name="unique_high_risk_med")

# b) Total count of high-risk medication prescriptions per admission
high_risk_med_count = prescriptions.groupby(["subject_id", "hadm_id"])["high_risk_med"] \
    .sum().reset_index(name="high_risk_med_count")

# c) Binary flag: 1 if any high-risk medication was prescribed, else 0.
high_risk_med_flag = prescriptions.groupby(["subject_id", "hadm_id"])["high_risk_med"] \
    .max().reset_index(name="high_risk_med_flag")

# --- 5. Merge the Metrics into a Single Summary DataFrame ---
high_risk_med_summary = high_risk_med_flag.merge(unique_high_risk_med, on=["subject_id", "hadm_id"], how="left") \
                                      .merge(high_risk_med_count, on=["subject_id", "hadm_id"], how="left")

# Fill missing unique_high_risk_count with 0 (for admissions with no high-risk meds)
high_risk_med_summary["unique_high_risk_med"] = high_risk_med_summary["unique_high_risk_med"].fillna(0).astype(int)


In [136]:
print(all_drugs.head())

   subject_id   hadm_id                                               drug
0    10000032  22595853  [furosemide, ipratropium bromide neb, furosemi...
1    10000032  22841357  [furosemide, rifaximin, sodium chloride 0.9%  ...
2    10000032  25742920  [sodium chloride 0.9%  flush, 0.9% sodium chlo...
3    10000032  29079034  [bisacodyl, bisacodyl, senna, calcium carbonat...
4    10000084  23052089  [pramipexole, pravastatin, rivastigmine, senna...


In [158]:
print("🔍  high_risk_med_counts:")
print(high_risk_med_summary["unique_high_risk_med"].value_counts(normalize=True)) 
print(high_risk_med_summary.head()) 

🔍  high_risk_med_counts:
unique_high_risk_med
1     0.210220
2     0.206627
0     0.196643
3     0.143022
4     0.092807
5     0.058902
6     0.036326
7     0.021836
8     0.012941
9     0.007811
10    0.004884
11    0.003080
12    0.001960
13    0.001256
14    0.000676
15    0.000442
16    0.000257
17    0.000149
18    0.000047
20    0.000032
19    0.000030
21    0.000024
22    0.000015
26    0.000004
23    0.000004
24    0.000004
Name: proportion, dtype: float64
   subject_id   hadm_id  high_risk_med_flag  unique_high_risk_med  \
0    10000032  22595853                   0                     0   
1    10000032  22841357                   0                     0   
2    10000032  25742920                   1                     2   
3    10000032  29079034                   1                     1   
4    10000084  23052089                   1                     1   

   high_risk_med_count  
0                    0  
1                    0  
2                    3  
3               

In [299]:
# Define Palliative Care ICD Codes in Diagnosis Table 
palliative_care_codes = ["Z515", "V667"]

# Create Palliative Care Flag Based on Diagnoses
diagnosis_counts["palliative_care_flag"] = diagnosis_counts["diagnosis_list"].apply(
    lambda x: 1 if any(code in palliative_care_codes for code in x) else 0
)

In [301]:
print(diagnosis_counts.duplicated(subset=["subject_id", "hadm_id"]).sum())


0


In [265]:
print("🔍 Palliative Care Flag Distribution:")
print(diagnosis_counts["palliative_care_flag"].value_counts(normalize=True))


🔍 Palliative Care Flag Distribution:
palliative_care_flag
0    0.976385
1    0.023615
Name: proportion, dtype: float64


In [267]:
# Define ICD codes for cognitive impairment (e.g., Dementia, Alzheimer's) 
# Alzheimer's Disease
cognitive_impairment_codes = [
    "G30", "G300", "G301", "G308", "G309",  # Alzheimer's disease
]
# Vascular Dementia
cognitive_impairment_codes += [
    "F0150", "F0151", "F01.A0", "F01A1", "F01B0", "F01B1", "F01C0", "F01C1",  # Vascular dementia
]
# Dementia in Other Diseases
cognitive_impairment_codes += [
    "F0280", "F0281"  # Dementia in other diseases classified elsewhere
]
# Unspecified Dementia
cognitive_impairment_codes += [
    "F03", "F0390", "F0391"  # Unspecified dementia
]
# Mild Cognitive Impairment
cognitive_impairment_codes += [
    "G31", "G3184"  # Mild cognitive impairment
]

In [303]:
# Create flag for cognitive impairment
diagnosis_counts["cognitive_impairment_flag"] = diagnosis_counts["diagnosis_list"].apply(
    lambda x: 1 if any(str(code) in cognitive_impairment_codes for code in x) else 0
) 

In [270]:
print("🔍 cognitive_impairment_flag Flag Distribution:")
print(diagnosis_counts["cognitive_impairment_flag"].value_counts(normalize=True))


🔍 cognitive_impairment_flag Flag Distribution:
cognitive_impairment_flag
0    0.976255
1    0.023745
Name: proportion, dtype: float64


In [305]:
# Define ICD Codes for Delirium
delirium_icd_codes = ["F05", "2930", "2931","F051","F050","F059" ]  # ICD-10 & ICD-9 Codes
# Ensure diagnosis_list is a list, and handle missing values
diagnosis_counts["delirium"] = diagnosis_counts["diagnosis_list"].apply(
    lambda x: 1 if isinstance(x, list) and any(code in delirium_icd_codes for code in x) else 0
)

In [307]:
#test merge issue
print(diagnosis_counts.duplicated(subset=["subject_id", "hadm_id"]).sum())


0


In [275]:
print(diagnosis_counts["delirium"].value_counts(normalize=True))

delirium
0    0.982277
1    0.017723
Name: proportion, dtype: float64


In [309]:
# Create an empty DataFrame with the key columns
core_data = pd.DataFrame(columns=["subject_id", "hadm_id"])


In [287]:
print(diagnoses.groupby(["subject_id", "hadm_id"]).size().describe())


count    545497.000000
mean         11.667320
std           7.625992
min           1.000000
25%           6.000000
50%          10.000000
75%          16.000000
max          57.000000
dtype: float64


In [311]:
# 1. Create core_data as the base using admissions and patients (one row per admission)
core_data = admissions.merge(patients, on="subject_id", how="inner")

# 2. Merge diagnosis_counts for key flags (palliative care, delirium, cognitive impairment)
core_data = core_data.merge(
    diagnosis_counts[["subject_id", "hadm_id", "palliative_care_flag", "delirium", "cognitive_impairment_flag","num_comorbidities"]],
    on=["subject_id", "hadm_id"],
    how="left"
)

# Fill missing values for these flags
core_data["palliative_care_flag"] = core_data["palliative_care_flag"].fillna(0).astype(int)
core_data["delirium"] = core_data["delirium"].fillna(0).astype(int)
core_data["cognitive_impairment_flag"] = core_data["cognitive_impairment_flag"].fillna(0).astype(int)

# 3. Merge diagnoses if needed
core_data = core_data.merge(diagnoses, on=["subject_id", "hadm_id"], how="left")

# 4. Merge ICU data
core_data = core_data.merge(icustays, on=["subject_id", "hadm_id"], how="left")
core_data = core_data.merge(icu_history, on="subject_id", how="left")
#  ICU vitals + other icu chart events 
core_data = core_data.merge(icu_pivot, on=["subject_id", "hadm_id"], how="left")

# 5. Merge high-risk medication summary and all_drugs
core_data = core_data.merge(high_risk_med_summary, on=["subject_id", "hadm_id"], how="left")
core_data = core_data.merge(all_drugs, on=["subject_id", "hadm_id"], how="left")

# 6. Merge lab results
core_data = core_data.merge(lab_pivot, on=["subject_id", "hadm_id"], how="left")



# (Optional) Fill additional missing values if needed
core_data["num_comorbidities"] = core_data["num_comorbidities"].fillna(0).astype(int)
core_data["prior_icu_admissions"] = core_data["prior_icu_admissions"].fillna(0).astype(int)


In [312]:
# Print the column names
print(core_data.columns)

# Display the first 5 rows
print(core_data.head())

# Alternatively, get a concise summary including non-null counts and dtypes
print(core_data.info())


Index(['subject_id', 'hadm_id', 'admission_type', 'admission_location',
       'discharge_location', 'insurance', 'marital_status', 'race',
       'ed_time_spent', 'los_hosp', 'gender', 'anchor_age', 'anchor_year',
       'palliative_care_flag', 'delirium', 'cognitive_impairment_flag',
       'num_comorbidities', 'icd_code', 'stay_id', 'last_careunit', 'los',
       'prior_icu_admissions', 'icu_hr', 'icu_map', 'icu_iabp', 'icu_rr',
       'icu_pao2', 'icu_hb', 'icu_paco2', 'icu_spo2', 'icu_peep', 'icu_wbc',
       'icu_chloride', 'icu_creatinine', 'icu_glucose', 'icu_sodium',
       'icu_gcs_eye', 'icu_temp', 'icu_ph', 'icu_fio2', 'icu_airway',
       'icu_vent_mode', 'icu_gcs_verbal', 'icu_gcs_motor', 'icu_base_excess',
       'icu_bun', 'icu_lactate', 'icu_potassium', 'icu_hco3', 'icu_plt',
       'icu_rass', 'high_risk_med_flag', 'unique_high_risk_med',
       'high_risk_med_count', 'drug', 'Alb', 'CRP', 'Hb', 'Plt', 'WBC'],
      dtype='object')
   subject_id   hadm_id admission_ty

In [316]:
#Linkage Data Quality 
df=core_data
#Count occurrences of each patient (subject_id)
patient_counts = df["subject_id"].value_counts()

# Display summary
print("🔍 Number of Unique Patients:", df["subject_id"].nunique())
print("🔍 Total Rows in Dataset:", df.shape[0])
print("🔍 Distribution of Patient Admissions:")
print(patient_counts.describe())  # Summary statistics for occurrences

🔍 Number of Unique Patients: 223452
🔍 Total Rows in Dataset: 6599888
🔍 Distribution of Patient Admissions:
count    223452.000000
mean         29.536044
std          56.473785
min           1.000000
25%           6.000000
50%          13.000000
75%          30.000000
max        2570.000000
Name: count, dtype: float64


In [283]:
# Step 4: Save Processed Data for ML Modeling
output_path = "D:/MIMIC-IV-Data-Pipeline/processed_data"
os.makedirs(output_path, exist_ok=True)
core_data.to_csv(os.path.join(output_path, "delirium_prediction_data_v6.csv.gz"), index=False, compression='gzip')

print("✅ Cleaned dataset saved successfully! One row per admission.")

✅ Cleaned dataset saved successfully! One row per admission.
